In [79]:
import pymysql
from sklearn.ensemble import RandomForestClassifier
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from pandasql import sqldf
from sklearn import metrics
import pickle
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix

In [80]:
columns_drop=["ID", "TEAM_HOME", "TEAM_AWAY", "Unnamed: 0",
            "FAVOURITE_IS_HOME", "FAVOURITE_IS_AWAY",
             "CORNERS_H_45_60", "CORNERS_A",
            "NUM_DANGATTACKSAWAY_L45", "NUM_DANGATTACKSHOME_L45", "NUM_ATTACKSHOME_L45",
             "AVG_CORNERS_LOSE_AWAY",
            "CORNERS_A_45_60",'CORNERS_H2>60','CORNERS_A2>60', 'TOTAL_CORNERS>60',
             "GOALS_A",
            "NUM_MATCHESAWAY_POSITIVE_L5", "NUM_SHOTSHOME_L45", "TOTAL_CORNERS"]

In [81]:
data = pd.read_csv("../FEATURE_ENGINEER/tablon_uppert_70_q3.csv").sort_values(by=["ID"], ascending=True)

data_U370 = pd.read_csv("../FEATURE_ENGINEER/tablon_uppert_70_q3.csv").sort_values(by=["ID"], ascending=True)\
    .drop(columns=columns_drop, axis=1)
data_U370 = data_U370[data_U370.DATE_MATCH>='2021-12-10']
y_U370 = data_U370[data_U370.DATE_MATCH>='2021-12-10'][["output"]].reset_index(drop=True)
data_U370 = data_U370.drop(columns=["output", "DATE_MATCH"], axis=1)

data_U463 = pd.read_csv("../FEATURE_ENGINEER/tablon_undert_63_q4.csv").sort_values(by=["ID"], ascending=True)\
    .drop(columns=columns_drop, axis=1)
data_U463 = data_U463[data_U463.DATE_MATCH>='2021-12-10']
y_U463 = data_U463[data_U463.DATE_MATCH>='2021-12-10'][["output"]].reset_index(drop=True)
data_U463 = data_U463.drop(columns=[ "output", "DATE_MATCH"], axis=1)

data_U557 = pd.read_csv("../FEATURE_ENGINEER/tablon_undert_57_q5.csv").sort_values(by=["ID"], ascending=True)\
    .drop(columns=columns_drop, axis=1)
data_U557 = data_U557[data_U557.DATE_MATCH>='2021-12-10']
y_U557 = data_U557[data_U557.DATE_MATCH>='2021-12-10'][["output"]].reset_index(drop=True)
data_U557 = data_U557.drop(columns=[ "output", "DATE_MATCH"], axis=1)

In [82]:
data_U370.columns

Index(['ODDS_HOME', 'ODDS_AWAY', 'DIF_ODDS', 'NUM_FOULSHOME_L45', 'NUM_ATTACKSAWAY_L45', 'NUM_SAVESHOME_L45', 'POSESSION_HOME_L45', 'NUM_TACKLEAWAY_L45', 'NUM_SHOTSAWAY_L45', 'DIF_POSESSION_L45', 'DIF_SHOTS_L45', 'DIF_DANGATTACKS_L45', 'SUM_SHOTS_L60', 'SUM_DANGATTACKS_L60', 'GOALS_H', 'IS_FAVOURITE_LOSING', 'CORNERS_H', 'TOTAL_CORNERS_45_60', 'CORNERS_DIV_SHOTS', 'CORNERS_DIV_DANGATTACKS', 'LAST_MINUTECORNER_H_L60', 'LAST_MINUTECORNER_A_L60', 'LAST_MINUTECORNER_L60', 'DIF_GOALS_L60', 'DIF_CORNERS_L60', 'SUM_CORNERS_L60', 'GOALS_ATTACKS_L45', 'GOALS/DANGATTACKS_L45', 'CORNERS_SHOTS_L45', 'AVG_CORNERSHOMED2_L3', 'AVG_CORNERSHOMER2_L3', 'AVG_CORNERSAWAYD2_L3', 'AVG_CORNERSAWAYR2_L3', 'AVG_CORNERSHOMED2_L5', 'AVG_CORNERSAWAY2_L5', 'AVG_CORNERSAWAYD2_L5', 'AVG_CORNERSHOMED1_L3', 'AVG_CORNERSHOMER1_L3', 'AVG_CORNERSAWAY1_L3', 'AVG_CORNERSAWAYD1_L3', 'AVG_CORNERSHOME1_L5', 'AVG_CORNERSHOMED1_L5', 'AVG_CORNERSHOMER1_L5', 'AVG_CORNERSAWAY1_L5', 'AVG_CORNERSAWAYD1_L5', 'AVG_CORNERSAWAYR1_L5',
 

In [83]:
model_under_3_70 = pickle.load(open("./Corners_under3_70/model_under3_70.pkl", "rb"))
model_under_4_63 = pickle.load(open("./Corners_under4_64/model_under4_63.pkl", "rb"))
model_under_5_57 = pickle.load(open("./Corners_under5_58/model_under5_58.pkl", "rb"))

In [84]:
predictions = data[data.DATE_MATCH>='2021-12-10'].copy().reset_index()
predictions["P_U370"] = pd.DataFrame(model_under_3_70.predict_proba(data_U370)[:,1]).iloc[:, 0]\
    .apply(lambda x: 1 if x>0.625 else 0)
predictions["P_U463"] = pd.DataFrame(model_under_4_63.predict_proba(data_U463)[:,1]).iloc[:, 0]\
    .apply(lambda x: 1 if x>0.7 else 0)
predictions["P_U557"] = pd.DataFrame(model_under_5_57.predict_proba(data_U557)[:,1]).iloc[:, 0]\
    .apply(lambda x: 1 if x>0.68 else 0)

In [85]:
predictions["P_U370"] = predictions[["P_U370", "TOTAL_CORNERS"]]\
    .apply(lambda x: x[0] if x[1]>3 and x[1] < 9 else 0, axis=1)
predictions["P_U463"] = predictions[["P_U463", "TOTAL_CORNERS"]]\
    .apply(lambda x: x[0] if x[1]>3 and x[1]<9 else 0, axis=1)
predictions["P_U557"] = predictions[["P_U557", "TOTAL_CORNERS"]]\
    .apply(lambda x: x[0] if x[1]>4 and x[1] <9 else 0, axis=1)

In [86]:
predictions["U370"] = y_U370
predictions["U463"] = y_U463
predictions["U557"] = y_U557

In [87]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
predictions = predictions[(predictions.P_U370==1) | (predictions.P_U463==1) | (predictions.P_U557==1)]
predictions[["ID", "TEAM_HOME", "TEAM_AWAY", "DATE_MATCH", "P_U370", "P_U463", "P_U557", "U370", "U463", "U557"]]


VVVVVVVVXXXVVXV

,ID,TEAM_HOME,TEAM_AWAY,DATE_MATCH,P_U370,P_U463,P_U557,U370,U463,U557
3,4SJdMYRc,Athletic Club,Sevilla,2021-12-11,0,0,1,0,0,1
8,CEU6lmIr,Sassuolo,Lazio,2021-12-12,0,0,1,1,1,1
10,CbAHHAtT,Osasuna,Barcelona,2021-12-12,1,1,1,1,1,1
12,EgkylPYL,Nantes,Lens,2021-12-10,0,0,1,1,1,1
15,IReSko49,Lille,Lyon,2021-12-12,0,0,1,1,1,1
20,O4dOjRl3,Brest,Montpellier,2021-12-11,0,0,1,0,0,1
22,OhAQwhOK,Greuther Furth,Union Berlin,2021-12-12,1,0,0,0,1,1
23,Qs85KCdA,Mallorca,Celta de Vigo,2021-12-10,0,0,1,1,1,0
28,WrilCl4j,Valencia,Elche,2021-12-11,1,0,0,0,0,0
31,fwKBm7Xl,Torino,Bolonia,2021-12-12,1,0,1,1,1,1


# 12/18

In [75]:
11/(11+4)*100

73.33333333333333